# Como construir un chatbot simple con memoria almacenada usando LangChain
* El chatbot podrá mantener una conversación
* Recordará interacciones anteriores
* Podrá almacenar memoria en una archivo JSON

## Truco para evitar las desagradables advertencias de desuso de LangChain

En este ejercicio, vamos a usar la cadena heredada de LangChain (LLMChain). Aunque funciona bien, se nos va a estar mostrando una advertencia de desuso. Para poder evitarlo, vamos a ingresar el siguiente código:

In [5]:
import warnings
from langchain._api import LangChainDeprecationWarning

warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

##### Implementación inicial

In [6]:
import os

from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


_ = load_dotenv(find_dotenv())
openai_api_key = os.environ['OPENAI_API_KEY']

# Creamos el modelo a utilizar
chatModel = ChatOpenAI(model="gpt-4o-mini")

# Creamos nuestro formateador de salida
output_parser = StrOutputParser()


In [7]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="Mi color favorito es el azul."),
]

#### Llamamos al modelo de Chat (LLM)

In [3]:
chatModel.invoke(messagesToTheChatbot)

AIMessage(content='¡El azul es un color hermoso! A menudo se asocia con la calma, la tranquilidad y la confianza. ¿Tienes algún tono de azul favorito o alguna razón especial por la que te gusta?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 14, 'total_tokens': 55, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-56a298df-8e23-43f6-8cf1-b84b66e64217-0', usage_metadata={'input_tokens': 14, 'output_tokens': 41, 'total_tokens': 55, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

##### Vamos a revisar si el Chatbot recuerda nuestro color favorito

In [4]:
chatModel.invoke([
    HumanMessage(content="¿Recuerdas cual es mi color favorito?")
])

AIMessage(content='Lo siento, pero no tengo la capacidad de recordar información personal de conversaciones anteriores. Si me dices cuál es tu color favorito, estaré encantado de recordarlo durante esta conversación.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 17, 'total_tokens': 53, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-9b249f54-5cf0-4235-a46e-975993343973-0', usage_metadata={'input_tokens': 17, 'output_tokens': 36, 'total_tokens': 53, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

* Como podemos ver el chatbot, no tiene la capacidad de recordar nuestro color favorito

### Agregando memoria a nuestro ChatBot

In [8]:
from langchain import LLMChain
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

# Ahora los paquetes para manejar la memoria
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory

In [9]:
# Vamos a crear la memoria del chatbot
memory = ConversationBufferMemory(
    chat_memory=FileChatMessageHistory('messages.json'), # Guardamos en un archivo json la memoria
    memory_key="messages",
    return_messages=True
)

In [18]:
# Creamos nuestro Prompt Template
#TODO: Validemos la implementación de ChatPromptTemplate
prompt = ChatPromptTemplate(
    input_variables=['content','messages'],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

In [19]:
# Construimos nuestra cadena
chain = LLMChain(
    llm=chatModel,
    prompt=prompt,
    memory=memory,
)

In [20]:
# Iniciamos a interactuar con el chatbot
chain.invoke("Hola!!")

{'content': 'Hola!!',
 'messages': [],
 'text': '¡Hola! ¿Cómo puedo ayudarte hoy?'}

In [21]:
chain.invoke("Mi nombre es Percy Tejada")

{'content': 'Mi nombre es Percy Tejada',
 'messages': [HumanMessage(content='Hola!!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='¡Hola! ¿Cómo puedo ayudarte hoy?', additional_kwargs={}, response_metadata={})],
 'text': '¡Encantado de conocerte, Percy! ¿En qué puedo ayudarte hoy?'}

In [22]:
chain.invoke("¿Cual es mi nombre?")

{'content': '¿Cual es mi nombre?',
 'messages': [HumanMessage(content='Hola!!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='¡Hola! ¿Cómo puedo ayudarte hoy?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Mi nombre es Percy Tejada', additional_kwargs={}, response_metadata={}),
  AIMessage(content='¡Encantado de conocerte, Percy! ¿En qué puedo ayudarte hoy?', additional_kwargs={}, response_metadata={})],
 'text': 'Tu nombre es Percy Tejada. ¿Hay algo más con lo que te pueda ayudar?'}

* El historial de la conversación se estará guardando en el archivo `messages.json`.
* Este es solo un ejemplo. En una aplicación real probablemente no guardemos la memoria en un archivo json.
* Recuerda que la ventana de contexto es limitada y esto afectará el costo de uso de la API ChatGPT.